In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
import numpy as np
import random

In [3]:
data_path = 'Datasets/fe_scaled_one_hot_full_2.csv'

In [4]:
df_fe_scaled_one_hot_full = pd.read_csv(data_path)

In [5]:
columns = df_fe_scaled_one_hot_full.columns

In [6]:
columns_set1 = ['FIRE_YEAR','DISCOVERY_DATE','DISCOVERY_DOY','DISCOVERY_TIME','LATITUDE','LONGITUDE',
                'SOURCE_SYSTEM_TYPE_FED','SOURCE_SYSTEM_TYPE_INTERAGCY','SOURCE_SYSTEM_TYPE_NONFED',
                'NWCG_REPORTING_AGENCY_BIA','NWCG_REPORTING_AGENCY_BLM','NWCG_REPORTING_AGENCY_BOR',
                'NWCG_REPORTING_AGENCY_DOD','NWCG_REPORTING_AGENCY_DOE','NWCG_REPORTING_AGENCY_FS',
                'NWCG_REPORTING_AGENCY_FWS','NWCG_REPORTING_AGENCY_IA','NWCG_REPORTING_AGENCY_NPS',
                'NWCG_REPORTING_AGENCY_ST/C&L','NWCG_REPORTING_AGENCY_TRIBE','FIRE_SIZE_CLASS_A','FIRE_SIZE_CLASS_B',
                'FIRE_SIZE_CLASS_C','FIRE_SIZE_CLASS_D','FIRE_SIZE_CLASS_E','FIRE_SIZE_CLASS_F','FIRE_SIZE_CLASS_G',
                'OWNER_DESCR_BIA','OWNER_DESCR_BLM','OWNER_DESCR_BOR','OWNER_DESCR_COUNTY','OWNER_DESCR_FOREIGN',
                'OWNER_DESCR_FWS','OWNER_DESCR_MISSING/NOT SPECIFIED','OWNER_DESCR_MUNICIPAL/LOCAL','OWNER_DESCR_NPS',
                'OWNER_DESCR_OTHER FEDERAL','OWNER_DESCR_PRIVATE','OWNER_DESCR_STATE','OWNER_DESCR_STATE OR PRIVATE',
                'OWNER_DESCR_TRIBAL','OWNER_DESCR_UNDEFINED FEDERAL','OWNER_DESCR_USFS','STATE_AK', 'STATE_AL', 
                'STATE_AR', 'STATE_AZ', 'STATE_CA', 'STATE_CO', 'STATE_CT', 'STATE_DC', 'STATE_DE', 'STATE_FL', 
                'STATE_GA', 'STATE_HI', 'STATE_IA', 'STATE_ID', 'STATE_IL', 'STATE_IN', 'STATE_KS', 'STATE_KY', 
                'STATE_LA', 'STATE_MA', 'STATE_MD', 'STATE_ME', 'STATE_MI', 'STATE_MN', 'STATE_MO', 'STATE_MS', 
                'STATE_MT', 'STATE_NC', 'STATE_ND', 'STATE_NE', 'STATE_NH', 'STATE_NJ', 'STATE_NM', 'STATE_NV', 
                'STATE_NY', 'STATE_OH', 'STATE_OK', 'STATE_OR', 'STATE_PA', 'STATE_PR', 'STATE_RI', 'STATE_SC', 
                'STATE_SD', 'STATE_TN', 'STATE_TX', 'STATE_UT', 'STATE_VA', 'STATE_VT', 'STATE_WA', 'STATE_WI', 
                'STATE_WV', 'STATE_WY']
# DISCOVERY_TIME,Discovery_DOY,LATITUDE,LONGITUTE, all state columns, 

columns_set2 = ['FIRE_YEAR','DISCOVERY_DATE','lat_lon_dist','lat_lon_angle',
                'SOURCE_SYSTEM_TYPE_FED','SOURCE_SYSTEM_TYPE_INTERAGCY','SOURCE_SYSTEM_TYPE_NONFED',
                'NWCG_REPORTING_AGENCY_BIA','NWCG_REPORTING_AGENCY_BLM','NWCG_REPORTING_AGENCY_BOR',
                'NWCG_REPORTING_AGENCY_DOD','NWCG_REPORTING_AGENCY_DOE','NWCG_REPORTING_AGENCY_FS',
                'NWCG_REPORTING_AGENCY_FWS','NWCG_REPORTING_AGENCY_IA','NWCG_REPORTING_AGENCY_NPS',
                'NWCG_REPORTING_AGENCY_ST/C&L','NWCG_REPORTING_AGENCY_TRIBE','FIRE_SIZE_CLASS_A','FIRE_SIZE_CLASS_B',
                'FIRE_SIZE_CLASS_C','FIRE_SIZE_CLASS_D','FIRE_SIZE_CLASS_E','FIRE_SIZE_CLASS_F','FIRE_SIZE_CLASS_G',
                'OWNER_DESCR_BIA','OWNER_DESCR_BLM','OWNER_DESCR_BOR','OWNER_DESCR_COUNTY','OWNER_DESCR_FOREIGN',
                'OWNER_DESCR_FWS','OWNER_DESCR_MISSING/NOT SPECIFIED','OWNER_DESCR_MUNICIPAL/LOCAL','OWNER_DESCR_NPS',
                'OWNER_DESCR_OTHER FEDERAL','OWNER_DESCR_PRIVATE','OWNER_DESCR_STATE','OWNER_DESCR_STATE OR PRIVATE',
                'OWNER_DESCR_TRIBAL','OWNER_DESCR_UNDEFINED FEDERAL','OWNER_DESCR_USFS','DISCOVERY_TIME_BIN_A', 
                'DISCOVERY_TIME_BIN_E', 'DISCOVERY_TIME_BIN_EM','DISCOVERY_TIME_BIN_LN', 'DISCOVERY_TIME_BIN_M',
                'DISCOVERY_TIME_BIN_N','DISCOVERY_DOY_BIN_FALL','DISCOVERY_DOY_BIN_SPRING','DISCOVERY_DOY_BIN_SUMMER',
                'DISCOVERY_DOY_BIN_WINTER', 'REGION_East','REGION_Mid_West', 'REGION_Mountain_West',
                'REGION_North_East','REGION_North_West', 'REGION_Pacific_West', 'REGION_South','REGION_South_East']

In [6]:
for column in df_fe_scaled_one_hot_full.columns:
  if df_fe_scaled_one_hot_full[column].isna().sum()!=0:
    print(column) #no column with na values...good!

In [7]:
#do not need below lines because we are choosing columns according to the requirements from above cell
# df_dropped_state = df_fe_scaled_one_hot_full.copy()
# state_columns = [col for col in df_dropped_state.columns if 'state_' in col.lower()]
# df_dropped_state = df_dropped_state.drop(columns = state_columns)

In [16]:
# X = df_fe_scaled_one_hot_full[columns_set1]
X2 = df_fe_scaled_one_hot_full[columns_set2]
Y = df_fe_scaled_one_hot_full['STAT_CAUSE_DESCR_Arson']


In [9]:
print(len(Y))
print(sum(Y))

1880465
281455


Only 15% of the data is labelled 1. So, let us use F1 score as accuracy metric to account for data imbalance

First, check the baselines once again with the normal data

In [9]:
x_train,x_valid,y_train,y_valid = train_test_split(X,Y,test_size = 0.2)
# x_train2,x_valid2,y_train,y_valid = train_test_split(X2,Y,test_size = 0.2)

In [11]:
histgb = HistGradientBoostingClassifier().fit(x_train, y_train)
y_pred = histgb.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.8725634351078058

In [13]:
RFC = RandomForestClassifier(max_depth=50, random_state=0,bootstrap=True, n_jobs = -1).fit(x_train, y_train)
y_pred = RFC.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.8879745169412885

In [14]:
MLP_classifier = MLPClassifier(solver = 'lbfgs',alpha = 1e-2, hidden_layer_sizes = (64,32,32),
                               random_state = 1,verbose=False, max_iter=200,activation = 'relu')
MLP_classifier.fit(x_train,y_train)
y_pred= MLP_classifier.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)
accuracy


/Users/rishitajain/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8686388738955524

Now checking how the baselines look like with feature engineered data

In [10]:
histgb = HistGradientBoostingClassifier().fit(x_train2, y_train)
y_pred = histgb.predict(x_valid2)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.8696093785313739

In [13]:
RFC = RandomForestClassifier(max_depth=50, random_state=0,bootstrap=True, n_jobs =-1).fit(x_train2, y_train)
y_pred = RFC.predict(x_valid2)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.8821355356254968

In [ ]:
# We will not be using MLP with feature engineering data by assuming that feature engineering is not needed for MLP
# MLP_classifier = MLPClassifier(solver = 'lbfgs',alpha = 1e-2, hidden_layer_sizes = (64,32,32),
#                                random_state = 1,verbose=False, max_iter=200,activation = 'relu')
# MLP_classifier.fit(x_train2,y_train)
# y_pred= MLP_classifier.predict(x_valid)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

Now we will do grid search cv to find best parameters for each of the below classifiers
First, grid search cv for histGradientBoostClassifier
Second, RandomForestClassifier
Third, Neural Network
Fourth, ExtraTreesClassifier
Fifth, Decision Tree CV with VotingClassifier

In [14]:
# X = df_dropped_state.copy()
# cause_columns = [col for col in df_dropped_state.columns if 'stat_cause_' in col.lower()]
# X = X.drop(columns = cause_columns)
# Y = df_dropped_state['STAT_CAUSE_DESCR_Arson']

In [19]:
# histgb = HistGradientBoostingClassifier(max_depth=50).fit(x_train, y_train)
# y_pred = histgb.predict(x_valid)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

0.8701189291339994

In [201]:
# parameters = {#'loss':['log_loss', 'auto', 'binary_crossentropy', 'categorical_crossentropy'],
#                 'loss':['binary_crossentropy'],
# #               'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.2,0.3,0.4,0.5]},
#                 'learning_rate':[0.2],
# #                 'max_leaf_nodes':[5,10,15,20,25,30,35,40,45,50]}
#                 'max_leaf_nodes':[45],
# #               'max_iter':[100],
# #               'max_depth':[5,10,15,20,25,30,35,40,50,60,70,None]}
#                 'max_depth':[70],
# #               'min_samples_leaf':[5,10,15,20,25,30,35,40,50,60,80,100]}
#                 'min_samples_leaf':[100]}
# #               'l2_regularization':[0]}
# parameters

{'loss': ['binary_crossentropy'],
 'learning_rate': [0.2],
 'max_leaf_nodes': [45],
 'max_depth': [70],
 'min_samples_leaf': [35]}

In [202]:
# histgb = HistGradientBoostingClassifier()
# # clf = GridSearchCV(histgb, parameters)
# histgb_results = clf.fit(x_train,y_train)

In [203]:
# histgb_result = pd.DataFrame.from_dict(histgb_results.cv_results_)
# histgb_result[histgb_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_max_leaf_nodes,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.281308,0.486512,0.0987,0.013356,0.2,binary_crossentropy,70,45,35,"{'learning_rate': 0.2, 'loss': 'binary_crossen...",0.850263,0.850263,0.850263,0.850263,0.850263,0.850263,2.438086e-07,1


In [233]:
# histgb = HistGradientBoostingClassifier(loss='binary_crossentropy',learning_rate=0.2,max_leaf_nodes=45,
#                                         max_depth=70,min_samples_leaf=100)
# histgb.fit(x_train,y_train)
# y_pred = histgb.predict(x_valid)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

0.8505821698356524

In [ ]:
#hist gradient boosting classifier on feature engineered data

In [230]:
# parameters = {#'loss':['log_loss', 'auto', 'binary_crossentropy', 'categorical_crossentropy'],
#                 'loss':['binary_crossentropy'],
# #               'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.2,0.3,0.4,0.5]},
#                 'learning_rate':[0.5],
# #                 'max_leaf_nodes':[5,10,15,20,25,30,35,40,45,50]}
#                 'max_leaf_nodes':[300],
# #               'max_iter':[100],
# #               'max_depth':[5,10,15,20,25,30,35,40,50,60,70,None]}
#                 'max_depth':[100],
# #               'min_samples_leaf':[5,10,15,20,25,30,35,40,50,60,80,100]}
#                 'min_samples_leaf':[120]}
# #               'l2_regularization':[0]}
# parameters

{'loss': ['binary_crossentropy'],
 'learning_rate': [0.5],
 'max_leaf_nodes': [300],
 'max_depth': [100],
 'min_samples_leaf': [120]}

In [228]:
# histgb = HistGradientBoostingClassifier()
# clf = GridSearchCV(histgb, parameters)
# histgb_results = clf.fit(x_train2,y_train)

In [229]:
# histgb_result = pd.DataFrame.from_dict(histgb_results.cv_results_)
# histgb_result[histgb_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_max_leaf_nodes,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,28.226817,2.016156,0.713377,0.047181,0.5,binary_crossentropy,100,300,120,"{'learning_rate': 0.5, 'loss': 'binary_crossen...",0.881017,0.882067,0.880754,0.880781,0.880365,0.880997,0.000575,1


In [232]:
# histgb = HistGradientBoostingClassifier(loss='binary_crossentropy',learning_rate=0.5,max_leaf_nodes=300,
#                                         max_depth=100,min_samples_leaf=120)
# histgb.fit(x_train2,y_train)
# y_pred = histgb.predict(x_valid2)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

0.88260616390095

In [19]:
# x_train,x_valid,y_train,y_valid = train_test_split(X,Y,test_size = 0.1)
x_train2,x_valid2,y_train,y_valid = train_test_split(X2,Y,test_size = 0.1)

In [ ]:
#RandomForestClassifier on normal data #baseline is 0.887

In [27]:
parameters = {
    'criterion':['entropy'], #0.871
    'max_depth' : [100],
    'bootstrap' : [True], #871
#     'max_depth':[20],
    'min_samples_split':[15,20],#0.872
    'min_samples_leaf':[1,5],
    'max_features':['sqrt', 'log2', None, 0.7, 0.8],
    'n_jobs':[-1]
}

In [29]:
rfc_result = pd.DataFrame.from_dict(rfc_results.cv_results_)
rfc_result[rfc_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
13,19.454661,0.23337,0.165987,0.012336,True,entropy,100,0.7,1,20,-1,"{'bootstrap': True, 'criterion': 'entropy', 'm...",0.876416,0.876413,0.877343,0.874658,0.875269,0.87602,0.000947,1


Testing the final model on normal dataset

In [11]:
RFC = RandomForestClassifier(criterion='entropy',max_depth=100,bootstrap=True,min_samples_split=5,min_samples_leaf=1,
                            max_features=0.7,n_jobs=-1,random_state=0)

RFC.fit(x_train,y_train)

y_pred = RFC.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)

In [12]:
accuracy

0.8926438603115179

In [15]:
f1score = f1_score(y_valid,y_pred,average='weighted')
f1score

0.8836866046622843

In [33]:
parameters = {
    'criterion':['gini'], #0.871
    'max_depth' : [30],
    'bootstrap' : [True], #871
#     'max_depth':[20],
    'min_samples_split':[10],#0.872
    'min_samples_leaf':[1],
    'max_features':['sqrt', 'log2', None, 0.7, 0.8, 0.9],
    'n_jobs':[-1]
}

In [ ]:
RFC = RandomForestClassifier()
RFC_clf = GridSearchCV(RFC, parameters, verbose = 3)
rfc_results = RFC_clf.fit(x_train2,y_train)

In [35]:
rfc_result = pd.DataFrame.from_dict(rfc_results.cv_results_)
rfc_result[rfc_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,13.209311,0.221306,0.156824,0.0033,True,gini,30,0.7,1,10,-1,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.874661,0.877769,0.875562,0.878061,0.876173,0.876445,0.001296,1


In [20]:
RFC = RandomForestClassifier(criterion='gini',max_depth=30,bootstrap=True,min_samples_split=10,min_samples_leaf=1,
                            max_features=0.7,n_jobs=-1,random_state=0)

RFC.fit(x_train2,y_train)

y_pred = RFC.predict(x_valid2)
accuracy = accuracy_score(y_valid,y_pred)

In [21]:
accuracy

0.8925375039218919

In [22]:
f1score = f1_score(y_valid,y_pred,average='weighted')
f1score

0.8822642878092349